In [21]:
import pandas as pd
import numpy as np
import nltk
import sklearn as sk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
import re
from bs4 import BeautifulSoup
import contractions
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import platform 
import torch
import sys

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/geetshingi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/geetshingi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/geetshingi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [22]:
has_gpu = torch.cuda.is_available()
has_mps = getattr(torch,'has_mps',False)
device = "mps" if getattr(torch,'has_mps',False) \
    else "gpu" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
print(f"Target device is {device}")
    
import warnings 
warnings.filterwarnings("ignore")

Python Platform: macOS-13.2.1-arm64-arm-64bit
PyTorch Version: 2.0.0.dev20230212

Python 3.9.16 | packaged by conda-forge | (main, Feb  1 2023, 21:38:11) 
[Clang 14.0.6 ]
Pandas 1.5.3
Scikit-Learn 1.2.1
GPU is NOT AVAILABLE
MPS (Apple Metal) is AVAILABLE
Target device is mps


In [23]:
df = pd.read_csv('text_data_nathan.csv', usecols = ['Utterance', 'Label'])
df.head()

,Utterance,Label
0,Okay.,"fo_o_fw_""_by_bc"
1,"All right, uh,",b
2,feelings on what caused the S and L crisis,sd
3,I guess I don't have a real technical knowledg...,sd
4,I gather that there where large numbers of sit...,sd


In [24]:
# df['Label'] = np.where((df['Label'] != "b") & (df['Label'] != "sd") & (df['Label'] != "sv") & (df['Label'] != "%") & (df['Label'] != "aa"), "Other", df['Label'])
# df.head()

In [25]:
df['Label'].value_counts()

sd                 75136
b                  38281
sv                 26421
%                  15195
aa                 11123
ba                  4757
qy                  4725
ny                  3030
fc                  2581
qw                  1976
nn                  1374
bk                  1306
h                   1226
qy^d                1218
bh                  1053
^q                   983
bf                   952
fo_o_fw_"_by_bc      879
na                   847
ad                   745
^2                   723
b^m                  687
qo                   656
qh                   575
^h                   556
ar                   344
ng                   302
br                   298
no                   284
fp                   225
qrr                  209
arp_nd               207
t3                   117
oo_co_cc             110
aap_am               104
t1                   103
bd                   103
^g                    92
qw^d                  80
fa                    79


In [26]:
df['Utterance'] = df['Utterance'].str.lower()
df.head()

,Utterance,Label
0,okay.,"fo_o_fw_""_by_bc"
1,"all right, uh,",b
2,feelings on what caused the s and l crisis,sd
3,i guess i don't have a real technical knowledg...,sd
4,i gather that there where large numbers of sit...,sd


In [7]:
def remove_html(x):
    return BeautifulSoup(x,'html.parser').get_text(separator=" ")

df['Utterance'] = df['Utterance'].apply(lambda x: remove_html(x))
df.head()

,Utterance,Label
0,okay.,"fo_o_fw_""_by_bc"
1,"all right, uh,",b
2,feelings on what caused the s and l crisis,sd
3,i guess i don't have a real technical knowledg...,sd
4,i gather that there where large numbers of sit...,sd


In [8]:
df['Utterance'] = df['Utterance'].str\
                    .replace('[^a-zA-Z ]', '',regex=True)
df['Utterance'] = df['Utterance'].str\
                    .replace('http\S+|www.\S+', '',regex=True)
df.head()

,Utterance,Label
0,okay,"fo_o_fw_""_by_bc"
1,all right uh,b
2,feelings on what caused the s and l crisis,sd
3,i guess i dont have a real technical knowledge...,sd
4,i gather that there where large numbers of sit...,sd


In [27]:
df['Utterance'] = df['Utterance'].apply(lambda x:contractions.fix(x))
df.head()

,Utterance,Label
0,okay.,"fo_o_fw_""_by_bc"
1,"all right, uh,",b
2,feelings on what caused the s and l crisis,sd
3,i guess i do not have a real technical knowled...,sd
4,i gather that there where large numbers of sit...,sd


In [28]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df['Label'])
df['Label'] = le.transform(df['Label'])

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

shuffled_df = shuffle(df)

train_utterances, test_utterances, y_train, y_test = train_test_split(
                    shuffled_df['Utterance'], shuffled_df['Label'], 
                    train_size = 0.95, test_size = 0.05, shuffle=True, 
                    stratify = shuffled_df['Label'])

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer 

vect = TfidfVectorizer().fit(train_utterances)

train_utterances_tfidf = vect.transform(train_utterances) 
test_utterances_tfidf = vect.transform(test_utterances)

In [31]:
model = Perceptron()

model.fit(train_utterances_tfidf, y_train)
pred = model.predict(test_utterances_tfidf)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

,precision,recall,f1-score,support
0,0.654618,0.643421,0.648971,760.000000
1,0.000000,0.000000,0.000000,36.000000
2,0.000000,0.000000,0.000000,5.000000
3,0.466667,0.500000,0.482759,28.000000
4,0.045977,0.081633,0.058824,49.000000
5,0.442553,0.187050,0.262958,556.000000
6,0.000000,0.000000,0.000000,5.000000
7,0.166667,0.054054,0.081633,37.000000
8,0.048387,0.352941,0.085106,17.000000
9,0.000000,0.000000,0.000000,10.000000


In [32]:
from imblearn.pipeline import Pipeline as imbpipeline


pipeline = imbpipeline(steps = [['classifier', LinearSVC()]])

param_grid = {'classifier__C': [0.01, 0.1, 1, 10, 100, 1000],
              'classifier__penalty': ['l1', 'l2'],
              'classifier__loss': ['hinge', 'squared_hinge'],
             'classifier__class_weight': [None]}


l_svc = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_weighted',
                           cv=10,
                           n_jobs=-1,
                            verbose = 1,
                            refit = True)

l_svc.fit(train_utterances_tfidf, y_train)
pred = l_svc.predict(test_utterances_tfidf)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

Fitting 10 folds for each of 24 candidates, totalling 240 fits


/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/geetshin

/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/geetshin

,precision,recall,f1-score,support
0,0.746207,0.711842,0.728620,760.000000
1,0.000000,0.000000,0.000000,36.000000
2,0.000000,0.000000,0.000000,5.000000
3,0.680000,0.607143,0.641509,28.000000
4,0.000000,0.000000,0.000000,49.000000
5,0.519435,0.264388,0.350417,556.000000
6,0.000000,0.000000,0.000000,5.000000
7,0.545455,0.162162,0.250000,37.000000
8,0.000000,0.000000,0.000000,17.000000
9,0.000000,0.000000,0.000000,10.000000


In [33]:
# model = LinearSVC()

# model.fit(train_utterances_tfidf, y_train)
# pred = model.predict(test_utterances_tfidf)
# pred = classification_report(y_test, pred, output_dict=True)
# report = pd.DataFrame(pred).transpose()
# report

In [34]:
from imblearn.pipeline import Pipeline as imbpipeline


pipeline = imbpipeline(steps = [['classifier', LogisticRegression()]])

param_grid = {'classifier__C': np.logspace(-4, 4, 50),
              'classifier__penalty': ['l1', 'l2'],
             'classifier__class_weight': [None]}


lg = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_weighted',
                           cv=10,
                           n_jobs=-1,
                            verbose = 1,
                            refit = True)

lg.fit(train_utterances_tfidf, y_train)
pred = lg.predict(test_utterances_tfidf)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

...

/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

,precision,recall,f1-score,support
0,0.685552,0.636842,0.660300,760.000000
1,0.000000,0.000000,0.000000,36.000000
2,0.000000,0.000000,0.000000,5.000000
3,0.607143,0.607143,0.607143,28.000000
4,0.000000,0.000000,0.000000,49.000000
5,0.536184,0.293165,0.379070,556.000000
6,0.000000,0.000000,0.000000,5.000000
7,0.533333,0.216216,0.307692,37.000000
8,0.000000,0.000000,0.000000,17.000000
9,0.000000,0.000000,0.000000,10.000000


In [35]:
# model = LogisticRegression()
# model.fit(train_utterances_tfidf, y_train)
# pred = model.predict(test_utterances_tfidf)
# pred = classification_report(y_test, pred, output_dict=True)
# report = pd.DataFrame(pred).transpose()
# report

In [36]:
model = MultinomialNB()
model.fit(train_utterances_tfidf, y_train)
pred = model.predict(test_utterances_tfidf)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

,precision,recall,f1-score,support
0,0.863636,0.175000,0.291028,760.000000
1,0.000000,0.000000,0.000000,36.000000
2,0.000000,0.000000,0.000000,5.000000
3,0.000000,0.000000,0.000000,28.000000
4,0.000000,0.000000,0.000000,49.000000
5,0.513619,0.237410,0.324723,556.000000
6,0.000000,0.000000,0.000000,5.000000
7,0.000000,0.000000,0.000000,37.000000
8,0.000000,0.000000,0.000000,17.000000
9,0.000000,0.000000,0.000000,10.000000


In [37]:
from sklearn.utils import class_weight
classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

In [38]:
# from imblearn.pipeline import Pipeline as imbpipeline
# import xgboost as xgb


# pipeline = imbpipeline(steps = [['classifier', xgb.XGBClassifier()]])

# param_grid = {'classifier__max_depth': range (2, 10, 1),
#             'classifier__n_estimators': range(60, 220, 40),
#             'classifier__learning_rate': [0.1, 0.01, 0.05]}


# xg = GridSearchCV(estimator=pipeline,
#                            param_grid=param_grid,
#                            scoring='f1_weighted',
#                            cv=10,
#                            n_jobs=-1,
#                             verbose = 1,
#                             refit = True)

# xg.fit(train_utterances_tfidf, y_train)
# pred = xg.predict(test_utterances_tfidf)
# pred = classification_report(y_test, pred, output_dict=True)
# report = pd.DataFrame(pred).transpose()
# report

In [39]:
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(train_utterances_tfidf, y_train)
pred = model.predict(test_utterances_tfidf)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

,precision,recall,f1-score,support
0,0.821918,0.710526,0.762174,760.000000
1,0.000000,0.000000,0.000000,36.000000
2,0.000000,0.000000,0.000000,5.000000
3,0.736842,0.500000,0.595745,28.000000
4,0.500000,0.020408,0.039216,49.000000
5,0.560372,0.325540,0.411832,556.000000
6,0.000000,0.000000,0.000000,5.000000
7,0.750000,0.243243,0.367347,37.000000
8,0.000000,0.000000,0.000000,17.000000
9,0.000000,0.000000,0.000000,10.000000


In [40]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(train_utterances_tfidf, y_train)
pred = model.predict(test_utterances_tfidf)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

,precision,recall,f1-score,support
0,0.762097,0.746053,0.753989,760.000000
1,0.000000,0.000000,0.000000,36.000000
2,0.000000,0.000000,0.000000,5.000000
3,0.888889,0.571429,0.695652,28.000000
4,0.000000,0.000000,0.000000,49.000000
5,0.545723,0.332734,0.413408,556.000000
6,0.000000,0.000000,0.000000,5.000000
7,0.833333,0.135135,0.232558,37.000000
8,0.000000,0.000000,0.000000,17.000000
9,0.000000,0.000000,0.000000,10.000000
